前几节介绍的LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，**AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。**本节我们介绍网络中的网络（NiN）[1]。它提出了另外一个思路，即**串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络**。

## NiN块

我们知道，卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维。回忆在[“多输入通道和多输出通道”](channels.ipynb)一节里介绍的$1\times 1$卷积层。它可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用$1\times 1$卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。图5.7对比了NiN同AlexNet和VGG等网络在结构上的主要区别。


<div align=center>
<img  src="../img/chapter05/5.8_nin.svg"/>
</div>
<div align=center>左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部</div>

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的$1\times 1$卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。

In [1]:
import tensorflow as tf
print(tf.__version__)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2.2.0


In [2]:
def nin_block(num_channels, kernel_size, strides, padding):
    blk = tf.keras.models.Sequential()
    blk.add(tf.keras.layers.Conv2D(num_channels, kernel_size,strides=strides,padding=padding,activation='relu'))
    blk.add(tf.keras.layers.Conv2D(num_channels,kernel_size=1, activation='relu'))
    blk.add(tf.keras.layers.Conv2D(num_channels,kernel_size=1, activation='relu'))  #为什么是两个？
    return blk

## NiN模型

NiN是在AlexNet问世不久后提出的。它们的卷积层设定有类似之处。NiN使用卷积窗口形状分别为$11\times 11$、$5\times 5$和$3\times 3$的卷积层，相应的输出通道数也与AlexNet中的一致。每个NiN块后接一个步幅为2、窗口形状为$3\times 3$的最大池化层。

除使用NiN块以外，NiN还有一个设计与AlexNet显著不同：NiN去掉了AlexNet最后的3个全连接层，取而代之地，NiN使用了**输出通道数等于标签类别数**的NiN块，**然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类**。**这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层**。NiN的这个设计的好处是**可以显著减小模型参数尺寸，从而缓解过拟合**。然而，该设计有时会造成获得有效模型的训练时间的增加。 

In [3]:
net = tf.keras.models.Sequential()
net.add(nin_block(96, kernel_size=11, strides=4, padding='valid'))  
net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
net.add(nin_block(256, kernel_size=5, strides=1, padding='same'))
net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
net.add(nin_block(384, kernel_size=3, strides=1, padding='same'))
net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
net.add(tf.keras.layers.Dropout(0.5))
net.add(nin_block(10, kernel_size=3, strides=1, padding='same'))
net.add(tf.keras.layers.GlobalAveragePooling2D())
net.add(tf.keras.layers.Flatten())

下面构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状。

In [4]:
X = tf.random.uniform((1,224,224,1))
for blk in net.layers:
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)

sequential_1 output shape:	 (1, 54, 54, 96)
max_pooling2d output shape:	 (1, 26, 26, 96)
sequential_2 output shape:	 (1, 26, 26, 256)
max_pooling2d_1 output shape:	 (1, 12, 12, 256)
sequential_3 output shape:	 (1, 12, 12, 384)
max_pooling2d_2 output shape:	 (1, 5, 5, 384)
dropout output shape:	 (1, 5, 5, 384)
sequential_4 output shape:	 (1, 5, 5, 10)
global_average_pooling2d output shape:	 (1, 10)
flatten output shape:	 (1, 10)


## 获取数据和训练模型

我们依然使用Fashion-MNIST数据集来训练模型。NiN的训练与AlexNet和VGG的类似，但这里使用的学习率更大。

In [5]:
import numpy as np

class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0,axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0,axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]
        
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.train_images[index],224,224,)
        return resized_images.numpy(), self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.test_images[index],224,224,)
        return resized_images.numpy(), self.test_labels[index]

batch_size = 128
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)

x_batch shape: (128, 224, 224, 1) y_batch shape: (128,)


In [6]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.06, momentum=0.3, nesterov=False)
optimizer = tf.keras.optimizers.Adam(lr=1e-7)
net.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

x_batch, y_batch = dataLoader.get_batch_train(batch_size)
net.fit(x_batch, y_batch)

4/4 [==============================] - 1s 153ms/step - loss: 5.7151 - accuracy: 0.0859


In [9]:
def train_nin():
#     net.load_weights("5.8_nin_weights.h5")
    epoch = 5
    num_iter = dataLoader.num_train//batch_size
    for e in range(epoch):
        for n in range(num_iter):
            x_batch, y_batch = dataLoader.get_batch_train(batch_size)
            net.fit(x_batch, y_batch)
            if n%20 == 0:
                net.save_weights("5.8_nin_weights.h5")

train_nin()

4/4 [==============================] - 1s 192ms/step - loss: 1.8677 - accuracy: 0.2266


4/4 [==============================] - 1s 198ms/step - loss: 1.7980 - accuracy: 0.2656


4/4 [==============================] - 1s 355ms/step - loss: 1.7461 - accuracy: 0.3359


4/4 [==============================] - 1s 351ms/step - loss: 1.7366 - accuracy: 0.2500


4/4 [==============================] - 1s 358ms/step - loss: 1.6254 - accuracy: 0.3984


...

4/4 [==============================] - 1s 207ms/step - loss: 1.5069 - accuracy: 0.4609


4/4 [==============================] - 1s 210ms/step - loss: 1.4580 - accuracy: 0.4453


4/4 [==============================] - 1s 209ms/step - loss: 1.4952 - accuracy: 0.4297


4/4 [==============================] - 1s 211ms/step - loss: 1.4989 - accuracy: 0.4922


4/4 [==============================] - 1s 199ms/step - loss: 1.5482 - accuracy: 0.3906


我们将训练好的参数读入，然后取测试数据计算测试准确率

In [10]:
net.load_weights("5.8_nin_weights.h5")

x_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(x_test, y_test, verbose=2)

63/63 - 5s - loss: 1.4535 - accuracy: 0.4940


[1.453540325164795, 0.49399998784065247]

效果这么差的么。。

## 小结

* NiN重复使用由卷积层和代替全连接层的$1\times 1$卷积层构成的NiN块来构建深层网络。
* NiN去除了容易造成过拟合的全连接输出层，而是将其替换成输出通道数等于标签类别数的NiN块和全局平均池化层。
* NiN的以上设计思想影响了后面一系列卷积神经网络的设计。

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_convolutional-neural-networks/nin.html)

